In [1]:
import math
import random
import os
from pathlib import Path 
import time
from tqdm import tqdm

from sklearn.model_selection import train_test_split

import data_reader
import pickle

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

### define model

In [2]:
class OneLayerNN(nn.Module):

    def __init__(self, input_dim=174, hidden_dim=128, output_dim=1):
        super(OneLayerNN, self).__init__()
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

### get data

In [3]:
arr = data_reader.read()
X = arr[:, :-2]
Y = arr[:, -2:]
X_train, X_test, Y_train, Y_test = \
train_test_split(X, Y, test_size=0.20, random_state=777)

train_size, input_dim = X_train.shape

### create model object and loss/optimizer calculator

In [8]:
one_layer_nn = OneLayerNN(input_dim=input_dim, output_dim=1)
one_layer_nn.parameters()

optimizer = optim.SGD(one_layer_nn.parameters(), lr=0.005)
optimizer.zero_grad()

criterion = nn.MSELoss()

In [9]:
torch.tensor(Y_train[:,0]).shape

torch.Size([480])

### train

In [10]:
print("Training CH")

label_index = 0

one_layer_nn.train()
start_train = time.time()
for epoch in range(1000):
    
    optimizer.zero_grad()
    inputs = torch.tensor(X_train).float()
    golds  = torch.tensor(Y_train[:,0]).float().unsqueeze(1)
    preds  = one_layer_nn(inputs)
    #print(golds.shape, preds.shape)
    loss = criterion(preds, golds)
    loss.backward()
    optimizer.step()
    
    if epoch % 100 == 0:
        print("Training time: {} for epoch {}, loss = {}".format(time.time() - start_train, epoch, loss))
    

Training CH
Training time: 0.0030014514923095703 for epoch 0, loss = 0.39887359738349915
Training time: 0.14499926567077637 for epoch 100, loss = 0.37999552488327026
Training time: 0.29100656509399414 for epoch 200, loss = 0.36451390385627747
Training time: 0.42999982833862305 for epoch 300, loss = 0.3490292727947235
Training time: 0.5739998817443848 for epoch 400, loss = 0.33325034379959106
Training time: 0.7170000076293945 for epoch 500, loss = 0.317617803812027
Training time: 0.8610000610351562 for epoch 600, loss = 0.30242782831192017
Training time: 0.9999985694885254 for epoch 700, loss = 0.2880434989929199
Training time: 1.1409986019134521 for epoch 800, loss = 0.2747819125652313
Training time: 1.283998966217041 for epoch 900, loss = 0.2627441883087158


### test

In [11]:
one_layer_nn.eval()
inputs = torch.tensor(X_test).float()
golds  = torch.tensor(Y_test[:,0]).float().unsqueeze(1)
preds  = one_layer_nn(inputs)
#print(golds.shape, preds.shape)
loss = criterion(preds, golds)
print(loss)

tensor(0.3169, grad_fn=<MseLossBackward>)
